## Liquid Specific Gravity Prediction of C7+ Fractions of Gas Samples From Molecular Weight (feat. PNA Composition)

<div class="highlights" id="key1">
    <div class="highlights-title">Summary To Save Your Time</div>
    <div class="highlights-content">Test</div>
</div>

## 1. Motivation

Characterizations of the C7+ fractions require three parameters: normal boiling point (T<sub>b</sub>), molecular weight (MW), and liquid specific gravity (SG<sub>liq</sub>). For crude oil sampe analysis, you will always have MW and SG<sub>liq</sub>, but realistically you never get T<sub>b</sub> of the plus fractions unless you specifically ask for distillation data (which you almost never do). Usually the T<sub>b</sub> values are estimated using T<sub>b</sub>-MW-SG<sub>liq</sub> correlation models. 

<strong><u>However, unlike crude oil analysis, gas sample analysis doesn't always come with liquid specific gravity,</u></strong> because it's a gas sample. It reports gas specific gravity instead, as in <a href="#fig-1" class="internal-link">Figure 1</a>. While some labs report provide calculated values for both SG<sub>liq</sub> and SG<sub>gas</sub>, many labs don't. Furthermore, for simple unextended gas analysis that reports only paraffinic compounds upto C6 or C7, the properties of the plus fractions assume some default ternary paraffinic mixture of n-hexane, n-heptane, and n-octane (usually 6:3:1 ratio), which results in underestimation of SG<sub>liq</sub> because it ignores the presence of naphthenic and aromatic fractions which have higher specific gravity than paraffins. This post provides backgrounds and guidance on how to estimate SG<sub>liq</sub> of plus fractions, such as C6+ or C7+, from molecular weight, using Paraffin-Naphthene-Aromatic (PNA) composition and Single-Carbon-Number (SCN) group correlations.

<div class="row full_screen_margin_40 mobile_responsive_plot_full_width" id="fig-1">
<div class="col"><img src="jupyter_images/liquid_sg_gas_analysis_1.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 1:</strong> Gas analysis sample taken from a separator operating @ 50 psig  &amp; 90F. The analysis is missing SG<sub>liq</sub>, which is required for characterization of the Heptanes Plus (C7+) fraction. While the analysis reports SG<sub>gas</sub> , this can't be directly used for process simulations. One can notice that the specific gravity reported in this analysis is relative to gas from <i>"(Air=1)"</i>. It this was SG<sub>liq</sub>, it would've been shown as <i>"(Water=1)"</i>.</p></div>

## 2. Impact of PNA composition on MW and SG<sub>liq</sub>

PNA composition indicates the ratio of each hydrocarbon group within the plus fractions. It's essentially a method to represent the plus fractions as a ternary mixture composed of paraffins, naphthenes, and aromatics. The reason that specifically these three are used are because first they are the primary hydrocarbon groups that make up most of unrefined crude oil and natural gas, and they show distinctively different characteristics from each other chemical formula and molecular structure. Skimming through <a href="#fig-2" class="internal-link">Figure 2</a>, <a href="#fig-3" class="internal-link">3</a>, <a href="#fig-4" class="internal-link">4</a>, and <a href="#fig-5" class="internal-link">5</a> may help you visually understand their distinctive properites. 

For the purposes of C6+ or C7+ characterizations, PNA composition matters because nathphenic and aromatic fractions have higher SG<sub>liq</sub> than paraffins at the same molecular weight (recall that process simulation for the plus fractions require both SG<sub>liq</sub> and MW), as can be seen in <a href="#fig-2" class="internal-link">Figure 2</a>. Developing a SG<sub>liq</sub> correlation from MW assuming pure paraffinic composition will result in underestimated values of SG<sub>liq</sub>.

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-2">
<div class="col"><img src="jupyter_images/liquid_sg_MW_sg_liq.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 2:</strong> Scatter plot of known compounds upto n-hexadecane (C16H34) listed in the GPA 2145 table<sup><a class="internal-link" href="#id2" id="id21">[2]</a></sup>.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (2)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
from sklearn.linear_model import LinearRegression
from pint import UnitRegistry

# Initialize a unit registry
ureg = UnitRegistry()

df = pd.read_csv('GPA 2145-16 Compound Properties Table - English - Truncated and PNA Identified.csv')

x = df['Molar Mass [g/mol]']
y = df['Liq. Relative Density @60F:1atm']

# Labeling for displaying texts
labels = ['methane', 'propane', 'n-butane', 'n-heptane', 'n-octane', 'n-decane', 'cyclohexane', 'cyclopentane', 'ethane',
'n-dodecane','n-tetradecane','n-hexadecane', 'methanol', 'naphthalene', 'hydrogen', 'n-hexane', 'n-pentane', 'isopentane', 
 'sulfur dioxide', 'hydrogen sulfide', 'toluene', 'benzene', 'm-xylene', 'hexylbenzene', 
         'propylene', '1-butene', 'isobutane', 'cyclooctane',
         ]
df['Display Text'] = df['Compound'].isin(labels)

BTEX_idx = df[df['Is BTEX'] == True].index
aromatic_idx = df[df['Is Aromatic'] == True].index
non_HC_idx = df[df['Is Hydrocarbon'] == False].index
hydroxyl_idx = df[df['Is Hydroxyl'] == True].index
paraffinic_idx = df[df['Is Paraffinic'] == True].index
naphethenic_idx = df[df['Is Naphthenic'] == True].index
other_idx = df[df['Others'] == True].index
whole_idx = list(df.index)

fig, ax = plt.subplots(figsize=(8, 4.5))

alpha = 1
_1 = ax.scatter(x.loc[paraffinic_idx], y.loc[paraffinic_idx], s=50, edgecolor='k', alpha=alpha, label='Paraffinic')
_2 = ax.scatter(x.loc[naphethenic_idx], y.loc[naphethenic_idx], s=50, edgecolor='k', alpha=alpha, label='Naphthenic')
_3 = ax.scatter(x.loc[aromatic_idx], y.loc[aromatic_idx], s=50, edgecolor='k', alpha=alpha, label='Aromatic/BTEX')
_4 = ax.scatter(x.loc[hydroxyl_idx], y.loc[hydroxyl_idx], s=50, edgecolor='k', alpha=alpha, label='Hydroxylic')
_5 = ax.scatter(x.loc[non_HC_idx], y.loc[non_HC_idx], s=50, edgecolor='k', alpha=alpha, label='Non-HCs')
_6 = ax.scatter(x.loc[other_idx], y.loc[other_idx], s=50, edgecolor='k', alpha=alpha, label='Other-HCs')

c1 = _1.get_facecolor()[0]
c2 = _2.get_facecolor()[0]
c3 = _3.get_facecolor()[0]
c4 = _4.get_facecolor()[0]
c5 = _5.get_facecolor()[0]
c6 = _6.get_facecolor()[0]

ax.legend(fontsize=9, ncol=3, loc='lower right')

texts = df['Compound']
for i, txt in enumerate(texts):
    if df['Display Text'].loc[i]:
        c = c5
        ha ='left'
        va = 'top'
        icr_y = 0.01

        if ha == 'left':
            icr_x = 3
        else:
            icr_x = -3

        if df['Is Paraffinic'].loc[i]:
            c = c1
            ha ='left'
            va = 'top'
            icr_y = -0.01
            icr_x = 0

        if df['Is Naphthenic'].loc[i]:
            c = c2
            ha ='right'
            va = 'top'
            icr_y = -0.01
        if df['Is Aromatic'].loc[i]:
            c = c3
            va = 'bottom'
            ha = 'center'
        if df['Is Hydroxyl'].loc[i]:
            c = c4
            va = 'bottom'
            ha = 'center'
        if df['Others'].loc[i]:
            c = c6
            va = 'bottom'
            ha = 'center'
            icr_y = 0.01

        ax.annotate(txt, (x.loc[i] + icr_x, y.iloc[i] + icr_y), fontsize=10, c=c, ha=ha, va=va)

#ax.set_xlim(-65, None)
ax.set_ylim(0, 1.1)
ax.set_ylim(0.4, 0.95)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ylim_top = ax.get_ylim()[1]
ax.axhline(ylim_top, color='white', linewidth=2)

ax.set_ylabel('Liquid Specific Gravity', fontsize=11)
ax.set_xlabel('Molecular Weight', fontsize=11)
ax.text(0.03, 0.08, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('MW vs. SG_{liq}')
plain_txt = ', for different compound groups'

"""
Run this code and re-import matplotlib in case Latex runtime error occurs:

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
"""

fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0.01, horizontalalignment='left', fontsize=12, y=0.96)
yloc = 0.875
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.01, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))
ax.annotate('Data source: GPA 2145-16', xy=(-0.11, -.12), xycoords='axes fraction', fontsize=9)

fig.tight_layout()

fig.savefig('liquid_sg_MW_sg_liq.png', dpi=300, bbox_inches='tight')
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-3">
<div class="col"><img src="jupyter_images/liquid_sg_ghv_gas_vs_mw.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 3:</strong> Scatter plot of known compounds upto n-hexadecane (C16H34) listed in the GPA 2145 table<sup><a class="internal-link" href="#id2" id="id22">[2]</a></sup>.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (3)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('GPA 2145-16 Compound Properties Table - English - Truncated and PNA Identified.csv')

# Labeling for displaying texts
labels = ['methane', 'propane', 'n-butane', 'n-heptane', 'n-octane', 'n-decane', 'cyclohexane', 'cyclopentane', 'ethane',
'n-dodecane','n-tetradecane','n-hexadecane', 'methanol', 'ethanol', 'naphthalene', 'isobutylcyclopentane', 'hydrogen',
 'sulfur dioxide', 'hydrogen sulfide', 'toluene', 'benzene', 'm-xylene', 'pentylbenzene', 'hexylbenzene',
         'propylene', '1-butene'
         ]
df['Display Text'] = df['Compound'].isin(labels)

BTEX_idx = df[df['Is BTEX'] == True].index
aromatic_idx = df[df['Is Aromatic'] == True].index
non_HC_idx = df[df['Is Hydrocarbon'] == False].index
hydroxyl_idx = df[df['Is Hydroxyl'] == True].index
paraffinic_idx = df[df['Is Paraffinic'] == True].index
naphethenic_idx = df[df['Is Naphthenic'] == True].index
other_idx = df[df['Others'] == True].index
whole_idx = list(df.index)

x = df['Molar Mass [g/mol]']
y = df['Gross Heating Value Ideal Gas [Btu/ft^3]']

##################################### Plotting #######################################

fig, ax = plt.subplots(figsize=(8, 4.5))

alpha = 1
_1 = ax.scatter(x.loc[paraffinic_idx], y.loc[paraffinic_idx], s=50, edgecolor='k', alpha=alpha, label='Paraffinic')
_2 = ax.scatter(x.loc[naphethenic_idx], y.loc[naphethenic_idx], s=50, edgecolor='k', alpha=alpha, label='Naphthenic')
_3 = ax.scatter(x.loc[aromatic_idx], y.loc[aromatic_idx], s=50, edgecolor='k', alpha=alpha, label='Aromatic/BTEX')
_4 = ax.scatter(x.loc[hydroxyl_idx], y.loc[hydroxyl_idx], s=50, edgecolor='k', alpha=alpha, label='Hydroxylic')
_5 = ax.scatter(x.loc[non_HC_idx], y.loc[non_HC_idx], s=50, edgecolor='k', alpha=alpha, label='Non-HCs')
_6 = ax.scatter(x.loc[other_idx], y.loc[other_idx], s=50, edgecolor='k', alpha=alpha, label='Other-HCs')

c1 = _1.get_facecolor()[0]
c2 = _2.get_facecolor()[0]
c3 = _3.get_facecolor()[0]
c4 = _4.get_facecolor()[0]
c5 = _5.get_facecolor()[0]
c6 = _6.get_facecolor()[0]

ax.legend(fontsize=9, ncol=3)

texts = df['Compound']
for i, txt in enumerate(texts):
    if df['Display Text'].loc[i]:
        c = c5
        ha ='left'
        va = 'top'
        
        if df['Is Paraffinic'].loc[i]: 
            c = c1
            ha ='right'
            va = 'bottom'
        if df['Is Naphthenic'].loc[i]:
            c = c2
            ha ='right'
            va = 'bottom'
        if df['Is Aromatic'].loc[i]:
            c = c3
            va = 'top'
            ha = 'left'
        if df['Is Hydroxyl'].loc[i]:
            c = c4
            va = 'bottom'
            ha = 'left'
        if df['Others'].loc[i]:
            c = c6
            va = 'top'
            ha = 'left'
        
        if ha == 'left':
            icr = 3
        else:
            icr= -3
        
        ax.annotate(txt, (x.loc[i] + icr, y.iloc[i]), fontsize=10, c=c, ha=ha, va=va)

ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
#ax.grid(axis='y', which='minor', linestyle='--', color='grey', alpha=0.2)
#ax.grid(axis='x', which='minor', color='grey', linestyle='--', alpha=0.2)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Molecular Weight', fontsize=11)
ax.set_ylabel('Gross Heating Value [Btu/scf]', fontsize=11)
ax.text(0.99, 0.1, 'aegis4048.github.io', fontsize=12, ha='right', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

#ax.set_xlim(0, 1)
#ax.set_ylim(10000, 30000)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('GHV_{gas} vs. MW')
plain_txt = ', for different compound groups'

fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0, horizontalalignment='left', fontsize=12, y=0.96)
yloc = 0.88
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.02, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

ax.annotate('Data source: GPA 2145-16', xy=(-0.11, -.12), xycoords='axes fraction', fontsize=9)

fig.tight_layout()

fig.savefig('liquid_sg_ghv_gas_vs_mw.png', dpi=300, bbox_inches='tight')
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-4">
<div class="col"><img src="jupyter_images/liquid_sg_Tb_sg_liq.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 4:</strong> Scatter plot of known compounds upto n-hexadecane (C16H34) listed in the GPA 2145 table<sup><a class="internal-link" href="#id2" id="id23">[2]</a></sup>.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (4)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
from sklearn.linear_model import LinearRegression
from pint import UnitRegistry

# Initialize a unit registry
ureg = UnitRegistry()

df = pd.read_csv('GPA 2145-16 Compound Properties Table - English - Truncated and PNA Identified.csv')

x = df['Boiling T. [F]']
y = df['Liq. Relative Density @60F:1atm']

BTEX_idx = df[df['Is BTEX'] == True].index
aromatic_idx = df[df['Is Aromatic'] == True].index
non_HC_idx = df[df['Is Hydrocarbon'] == False].index
hydroxyl_idx = df[df['Is Hydroxyl'] == True].index
paraffinic_idx = df[df['Is Paraffinic'] == True].index
naphethenic_idx = df[df['Is Naphthenic'] == True].index
other_idx = df[df['Others'] == True].index
whole_idx = list(df.index)

fig, ax = plt.subplots(figsize=(8, 4.5))

alpha = 1
_1 = ax.scatter(x.loc[paraffinic_idx], y.loc[paraffinic_idx], s=50, edgecolor='k', alpha=alpha, label='Paraffinic')
_2 = ax.scatter(x.loc[naphethenic_idx], y.loc[naphethenic_idx], s=50, edgecolor='k', alpha=alpha, label='Naphthenic')
_3 = ax.scatter(x.loc[aromatic_idx], y.loc[aromatic_idx], s=50, edgecolor='k', alpha=alpha, label='Aromatic/BTEX')
_4 = ax.scatter(x.loc[hydroxyl_idx], y.loc[hydroxyl_idx], s=50, edgecolor='k', alpha=alpha, label='Hydroxylic')
_5 = ax.scatter(x.loc[non_HC_idx], y.loc[non_HC_idx], s=50, edgecolor='k', alpha=alpha, label='Non-HCs')
_6 = ax.scatter(x.loc[other_idx], y.loc[other_idx], s=50, edgecolor='k', alpha=alpha, label='Other-HCs')

c1 = _1.get_facecolor()[0]
c2 = _2.get_facecolor()[0]
c3 = _3.get_facecolor()[0]
c4 = _4.get_facecolor()[0]
c5 = _5.get_facecolor()[0]
c6 = _6.get_facecolor()[0]

ax.legend(fontsize=9, ncol=3, loc='lower right')

labels = ['methane', 'propane', 'n-butane', 'n-heptane', 'n-octane', 'n-decane', 'cyclohexane', 'cyclopentane', 'ethane',
'n-dodecane','n-tetradecane','n-hexadecane', 'methanol', 'ethanol', 'naphthalene', 'hydrogen', 'cyclooctane',
 'sulfur dioxide', 'hydrogen sulfide', 'toluene', 'benzene', 'm-xylene', 'hexylbenzene',
         '1-butene', 'isobutane', 'n-pentane', 'isopentane', '2,2-dimethylpropane', 'n-hexane', 'n-nonane'
         ]
df['Display Text'] = df['Compound'].isin(labels)

texts = df['Compound']
for i, txt in enumerate(texts):
    if df['Display Text'].loc[i]:
        c = c5
        ha ='left'
        va = 'top'
        icr_y = 0.01

        if ha == 'left':
            icr_x = 3
        else:
            icr_x = -3

        if df['Is Paraffinic'].loc[i]:
            c = c1
            ha ='left'
            va = 'top'
            icr_y = -0.01
            icr_x = 0

        if df['Is Naphthenic'].loc[i]:
            c = c2
            ha ='right'
            va = 'top'
            icr_y = -0.01
        if df['Is Aromatic'].loc[i]:
            c = c3
            va = 'bottom'
            ha = 'center'
        if df['Is Hydroxyl'].loc[i]:
            c = c4
            va = 'bottom'
            ha = 'center'
        if df['Others'].loc[i]:
            c = c6
            va = 'top'
            ha = 'right'

        ax.annotate(txt, (x.loc[i] + icr_x, y.iloc[i] + icr_y), fontsize=10, c=c, ha=ha, va=va)

ax.set_xlim(-65, None)
ax.set_ylim(0.4, 0.95)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ylim_top = ax.get_ylim()[1]
ax.axhline(ylim_top, color='white', linewidth=2)

ax.set_ylabel('Liquid Specific Gravity', fontsize=11)
ax.set_xlabel('Normal Boiling Temperature [F]', fontsize=11)
ax.text(0.03, 0.08, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('T_{b} vs. SG_{liq}')
plain_txt = ', for different compound groups'

"""
Run this code and re-import matplotlib in case Latex runtime error occurs:

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
"""

fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0.01, horizontalalignment='left', fontsize=12, y=0.96)
yloc = 0.875
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.01, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))
ax.annotate('Data source: GPA 2145-16', xy=(-0.11, -.12), xycoords='axes fraction', fontsize=9)

fig.tight_layout()
fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-5">
<div class="col"><img src="jupyter_images/liquid_sg_ghv_liq_vs_sg_liq.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 5:</strong> Scatter plot of known compounds upto n-hexadecane (C16H34) listed in the GPA 2145 table<sup><a class="internal-link" href="#id2" id="id24">[2]</a></sup>.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (5)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('GPA 2145-16 Compound Properties Table - English - Truncated and PNA Identified.csv')

x = df['Gross Heating Value Ideal Gas [Btu/lbm]']
y = df['Liq. Relative Density @60F:1atm']

BTEX_idx = df[df['Is BTEX'] == True].index
aromatic_idx = df[df['Is Aromatic'] == True].index
non_HC_idx = df[df['Is Hydrocarbon'] == False].index
hydroxyl_idx = df[df['Is Hydroxyl'] == True].index
paraffinic_idx = df[df['Is Paraffinic'] == True].index
naphethenic_idx = df[df['Is Naphthenic'] == True].index
other_idx = df[df['Others'] == True].index
whole_idx = list(df.index)

fig, ax = plt.subplots(figsize=(8, 4.5))

alpha = 1
_1 = ax.scatter(x.loc[paraffinic_idx], y.loc[paraffinic_idx], s=50, edgecolor='k', alpha=alpha, label='Paraffinic')
_2 = ax.scatter(x.loc[naphethenic_idx], y.loc[naphethenic_idx], s=50, edgecolor='k', alpha=alpha, label='Naphthenic')
_3 = ax.scatter(x.loc[aromatic_idx], y.loc[aromatic_idx], s=50, edgecolor='k', alpha=alpha, label='Aromatic/BTEX')
_4 = ax.scatter(x.loc[hydroxyl_idx], y.loc[hydroxyl_idx], s=50, edgecolor='k', alpha=alpha, label='Hydroxylic')
_5 = ax.scatter(x.loc[non_HC_idx], y.loc[non_HC_idx], s=50, edgecolor='k', alpha=alpha, label='Non-HCs')
_6 = ax.scatter(x.loc[other_idx], y.loc[other_idx], s=50, edgecolor='k', alpha=alpha, label='Other-HCs')

c1 = _1.get_facecolor()[0]
c2 = _2.get_facecolor()[0]
c3 = _3.get_facecolor()[0]
c4 = _4.get_facecolor()[0]
c5 = _5.get_facecolor()[0]
c6 = _6.get_facecolor()[0]

ax.legend(fontsize=9, ncol=3, loc='upper right')

labels = ['methane', 'propane', 'n-butane', 'n-heptane', 'n-octane', 'n-decane', 'cyclohexane', 'cyclopentane', 'ethane',
'n-pentadecane', 'naphthalene', 'hydrogen', 'n-hexane', 'n-pentane',
 'sulfur dioxide', 'hydrogen sulfide', 'toluene', 'benzene', 'm-xylene','hexylbenzene', 'styrene',
         'propylene', '1-butene',
         ]
df['Display Text'] = df['Compound'].isin(labels)

texts = df['Compound']
for i, txt in enumerate(texts):
    if df['Display Text'].loc[i]:
        c = c5
        ha ='left'
        va = 'top'

        y_icr = 0
        if df['Is Paraffinic'].loc[i]:
            c = c1
            ha ='left'
            va = 'bottom'
        if df['Is Naphthenic'].loc[i]:
            c = c2
            ha ='right'
            va = 'top'
        if df['Is Aromatic'].loc[i]:
            c = c3
            va = 'bottom'
            ha = 'left'
        if df['Is Hydroxyl'].loc[i]:
            c = c4
            va = 'bottom'
            ha = 'left'
        if df['Others'].loc[i]:
            c = c6
            va = 'top'
            ha = 'right'

        if txt in ['benzene', 'styrene',]:
            ha = 'right'
        if txt in ['styrene']:
            va = 'bottom'
        if txt in ['toluene']:
            ha = 'center'
            va = 'top'
            y_icr = -0.01
        if txt in ['m-xylene']:
            ha = 'left'
            va = 'bottom'
            y_icr = 0.05

        if ha == 'left':
            icr = 100
        else:
            icr= -50

        ax.annotate(txt, (x.loc[i] + icr, y.iloc[i] + y_icr), fontsize=10, c=c, ha=ha, va=va)

ax.set_xlim(17000, 25000)
ax.set_ylim(0.2, 1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ylim_top = ax.get_ylim()[1]
ax.axhline(ylim_top, color='white', linewidth=2)

ax.set_xlabel('Liquid Gross Heating Value [Btu/lbm]', fontsize=11)
ax.set_ylabel('Liquid Specific Gravity', fontsize=11)
ax.text(0.03, 0.08, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('GHV_{liq} vs. SG_{liq}')
plain_txt = ', for different compound groups'

"""
Run this code and re-import matplotlib in case Latex runtime error occurs:

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
"""

fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0.01, horizontalalignment='left', fontsize=12, y=0.96)
yloc = 0.875
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.01, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))
ax.annotate('Data source: GPA 2145-16', xy=(-0.11, -.12), xycoords='axes fraction', fontsize=9)

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="alert alert-info" id="what_is_PNA">
    <h4>Notes: What are paraffins, naphthenes, and aromatics?</h4>
    <p>Paraffins, naphtehens, and aromatics are the primary hydrocarbon groups 
        that make up compositions of the naturally occuring crude oil and gas. The below is a brief on each of them: </p>    
    <p><strong>Paraffins:</strong> Known as alkanes, paraffins follow the general formula $C_nH_{2n+2}$. This group is characterized by its high hydrogen to carbon ratio, contributing to their higher energy density per mass compared to other hydrocarbon types. Examples include Methane (CH₄), Ethane (C₂H₆), Propane (C₃H₈), Butane (C₄H₁₀), Pentane (C₅H₁₂), Hexane (C₆H₁₄), Heptane (C₇H₁₆), Octane (C₈H₁₈), Nonane (C₉H₂₀), and Decane (C₁₀H₂₂).</p>
    <p><strong>Naphthenes:</strong> Also known as cycloalkanes, naphthenes are very similar to paraffins and follow the general chemical formula $C_nH_{2n}$. Compounds typically containing "cyclo" in their names are considered naphthenic. Examples are Cyclopropane (C₃H₆), Cyclobutane (C₄H₈), Cyclopentane (C₅H₁₀), Cyclohexane (C₆H₁₂), Cycloheptane (C₇H₁₄), Cyclooctane (C₈H₁₆), Cyclononane (C₉H₁₈), Cyclodecane (C₁₀H₂₀), Methylcyclopentane (C₆H₁₂), and Ethylcyclohexane (C₈H₁₆).</p>
    <p><strong>Aromatics:</strong> This class of hydrocarbons is characterized by one or more benzene rings in their molecular structure. The simplest aromatic compound, benzene, has the chemical formula $C_6H_6$. Aromatic compounds generally follow the formula $C_nH_n$ for monocyclic aromatics, but the presence of multiple rings and substituents can alter this formula. The carbon to hydrogen ratio for aromatics is generally 1:1, less than half of paraffins, explaining aromatics' lower energy density per mass. BTEX compounds fall within this class. Examples include Benzene (C₆H₆), Toluene (C₇H₈), Ethylbenzene (C₈H₁₀), o-Xylene (C₈H₁₀), m-Xylene (C₈H₁₀), p-Xylene (C₈H₁₀), Styrene (C₈H₈), Naphthalene (C₁₀H₈), Phenol (C₆H₅OH), and Aniline (C₆H₅NH₂).</p>
</div>

## 3. Problems with plus fractions in un-extended gas analysis

<div class="highlights" id="key1">
<div class="highlights-title">Summary</div>
<div class="highlights-content">C6+ fractions in simple un-extended gas analysis assume a fixed pure paraffinic pseudo-compound of 60% hexane, 30% heptane, and 10% octane. Because the fraction is assumed to be purely paraffinic (no naphthenics and aromatics), such ternary mixture results in underestimation of SG<sub>liq</sub> for C6+.</div>
</div>

For simple un-extended gas analysis, it is a common practice to assume a ternary composition of hexane, heptane, and octane to replace the plus fractions, because natural gas samples have negligible volumes of <i>"heavy-ends"</i> starting from n-nonane. Traditionally for C6+ fractions, the ratio is fixed at 60% n-hexane, 30% n-heptane, and 10% n-octane. Note that this ratio is arbitrary; the GPA 2286 standard<sup><a class="internal-link" href="#id3" id="id31">[3]</a></sup> states that <i>"Constant values for hexanes plus (C6+) or heptanes plus (C7+) if not actually determined by extended analysis should be
mutually agreed upon by all concerned parties."</i> Some reports explicitly show the assumed ratio, as shown in <a href="#fig-6" class="internal-link">Figure 6</a>. The calculated properites of the report is further validated through a process simulation software, as shown in <a href="#fig-7" class="internal-link">Figure 7</a>.

<div class="row full_screen_margin_100 mobile_responsive_plot_full_width" id="fig-6">
<div class="col"><img src="jupyter_images/liquid_sg_c6_assumed_ratio.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 6:</strong> Gas analysis of a sample taken at an atmospheric tank. It explicitly shows the assumed ternary mixture ratio of 6:3:1 for the Hexanes Plus (C6+) fraction.</p></div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-7">
<div class="col"><img src="jupyter_images/liquid_sg_C6_assumed_ratio_promax.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 7:</strong> BRE Promax screenshot, which replicates the above gas sample analysis for sanity check. Observe that the MW and specific gravity of the whole sample match exactly with the lab sample analysis <a href="#fig-6" class="internal-link">Figure 6</a>. This proves that the C6, C7, and C8 shown in the above samle analysis correspond to n-hexane, n-heptane, and n-octane.</p></div>

<div><hr></div>

Some labs don't explicity show the assumed ratio, as in <a href="#fig-8" class="internal-link">Figure 8</a>. However, the 6:3:1 rule for the C6+ fractions still holds for this sample. <a href="#fig-9" class="internal-link">Figure 9</a> shows a simulation result assuming the Hexanes Plus fraction being composed of 6:3:1 C6, C7, and C8. The molecular weight and gross heating values from the lab report (MW=93.189, GHV=5129.2) and the simulation result (MW=93.189, GHV=5129.2) match exactly with one another, confirming that the properties for the Hexanes Plus in the lab report are not measured, but stems from an arbitrarily assumed pure paraffinic composition with 6:3:1 ratio. 

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-8">
<div class="col"><img src="jupyter_images/liquid_sg_sample3.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 8:</strong> Unextended liquid analysis of a condensed natural gas liquid (NGL) taken from a pressurized midstream pipeline. While the report doesn't explicitly show the assumed ratio for the Hexanes Plus fraction, one can guess the 6:3:1 rule from the calculated MW of the Hexanes Plus fraction. </p></div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-9">
<div class="col"><img src="jupyter_images/liquid_sg_sample_3_promax.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 9:</strong> BRE Promax screenshot, which replicates the above gas sample analysis for sanity check. 6:3:1 ratio of hexane, heptane, and octane composition was assumed to replace the 60.179% Hexanes Plus fraction in <a href="#fig-8" class="internal-link">Figure 8</a>.</p></div>

<div><hr></div>

**Limitations with assuming 6:3:1 C6:C7:C8 ratio for C6+ fractions**

N-hexane, n-heptane, and n-octane are paraffinic compounds. As demonstrated in <a href="#fig-2" class="internal-link">Figure 2</a>, paraffinic compounds have lower SG<sub>liq</sub> compared to aromatic and naphthenic fractions. Since real-life plus fractions aren't composed of 100% paraffins, assuming pure paraffinic composition underestimates SG<sub>liq</sub> of the plus fractions. 

Consider the extended gas analysis upto n-decane (C10H22) in <a href="#fig-10" class="internal-link">Figure 10</a>. The analysis is provided by the same lab as in <a href="#fig-8" class="internal-link">Figure 8</a>. However, for this particular sample, extended analysis is applied, identifying virtually all possible hydrocarbon compounds that could naturally occur between C6 to C10. The Hexanes Plus fraction has MW=90.161, which can be matched assuming a ternary mixture of 77.27% hexane, 17.05% heptane, and 5.68% octane. The composition ratio was input into the simulation software, of which its result is shown in <a href="#fig-11" class="internal-link">Figure 11</a>. The measured lab reports have SG=0.7142 and GHV=4849.0, while the sim result of the assumed ternary mixture show SG=0.6692 and GHV=4968.0. The discrepency stems from ignoring the presence of aromatic and naphthenic fractions, which exhibit meaningful difference in MW vs. SG<sub>liq</sub> (<a href="#fig-2" class="internal-link">Figure 2</a>) and MW vs. GHV (<a href="#fig-3" class="internal-link">Figure 3</a>) relationships.

This exercise demonstrates that an accurate estimation of MW and SG<sub>liq</sub> of the C6+ or C7+ fractions require correlation models that takes PNA composition into account. 

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-10">
<div class="col"><img src="jupyter_images/liquid_sg_extended.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 10:</strong> Extended analysis upto C10 of a gas sample taken from an atmospheric tank. This time the properties of the Hexanes+ stem from physically measured mole fractions of compounds, rather than an assumed ternary mixture with 6:3:1 ratio. Note that the BTEX (aromatic) fraction make up 10.34% of the Hexanes Plus fraction (0.092/0.889 = 0.1034). Recall that the aromatic fractions have higher SG<sub>liq</sub> than paraffins at a given molecular weight.</p></div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-11">
<div class="col"><img src="jupyter_images/liquid_sg_extended_promax.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 11:</strong> Simulation result for the Hexanes Plus fraction of MW=90.161. Assuming a pure paraffinic fraction composed of n-hexane, n-heptane, and n-octane, 77.27%, 17.05%, and 5.68% gives the MW value matching 90.161. However, comparison with the actual lab analysis in <a href="#fig-10" class="internal-link">Figure 10</a> indicates some property discrepancies due to ignoring the presence of naphthenic and aromatic fractions. SG<sub>liq</sub> is underestimated to due paraffins having smaller SG<sub>liq</sub> at a given MW (<a href="#fig-2" class="internal-link">Figure 2</a>). On the other hand, gross heating value (GHV) is overestimated due to aromatics having smaller GHV than paraffins at a given MW (<a href="#fig-3" class="internal-link">Figure 3</a>).</p></div>

## 4. SG<sub>liq</sub> estimation from MW and SCN groups

Single-Carbon-Number (SCN) groups can be thought of as a representation of the plus fractions in relation to number of carbon atoms. For example, a plus fraction with MW=95 can be represented with SCN=7 according to <a href="#table-1" class="internal-link">Table 1</a>, which has T<sub>b</sub>=657R and sg=0.727 with the method of Riazi &amp; Al-Sahhaf (1996)<sup><a class="internal-link" href="#id5" id="id51">[5]</a></sup>. The table features two SCN models by Katz &amp; Firoozabadi (1978)<sup><a class="internal-link" href="#id4" id="id41">[4]</a></sup> and Riazi &amp; Al-Sahhaf (1996)<sup><a class="internal-link" href="#id5" id="id51">[5]</a></sup>, and the properties of normal paraffins with the corresponding carbon numbers. Both SCN models are developed to incorporate the effects of naphthenic and aromatic fractions, which show distinctively different properites compared to those of paraffins. Notably, <a href="#fig-12" class="internal-link">Figure 12</a> highlights the significant gap in SG<sub>liq</sub> of pure paraffins vs. SCN models due to the impacts of naphthenic and aromatic compositions, which are known to have highre SG<sub>liq</sub> at a given molecular weight, as shown in <a href="#fig-2" class="internal-link">Figure 2</a> above.

The method by Riazi & Al-Sahhaf claims to have improved on the previous Katz & Firoozabadi method. The paper also introduces numerical correlations for approximaing Tb and SG<sub>liq</sub> from MW, which are presented in <a href="#eq-1" class="internal-link">Eq-1</a> and <a href="#eq-2" class="internal-link">Eq-2</a>.

<div id="eq-1" style="font-size: 1rem;">
$$ T_{b} = 1080 - \text{exp}(6.97996 - 0.01964MW^{2/3})  \tag{1}$$
</div>

<div id="eq-2" style="font-size: 1rem;">
$$ SG_{\text{liq}} = 1.07 - \text{exp}(3.56073 - 2.93886MW^{0.1})  \tag{2}$$
</div>

<div class="eq-terms">
    <div class="row eq-terms-where">where</div>
    <div class="row">
        <div class="col-2">$MW$</div>
        <div class="col-1 max-width-3">:</div>
        <div class="col-9">molecular weight</div>
    </div>        
    <div class="row">
        <div class="col-2">$T_b$</div>
        <div class="col-1 max-width-3">:</div>
        <div class="col-9">mean average boiling point [°K].</div>
    </div>        
    <div class="row">
        <div class="col-2">$SG_{\text{liq}}$</div>
        <div class="col-1 max-width-3">:</div>
        <div class="col-9">liquid specific gravity</div>
    </div>
</div>

However, <a href="#eq-2" class="internal-link">Eq-2</a> (blue line plot) fails to predict SG<sub>liq</sub> properly for MW<136 in <a href="#fig-12" class="internal-link">Figure 12</a>. This is highly problematic for gas sample modeling because the molecular weight of the plus fractions of gas sample usually falls in the range of 85~105. In his later publication in 2005<sup><a class="internal-link" href="#id1" id="id11">[1]</a>, chap 4.3</sup>, Riazi mentions that the model is valid for Nc &#8805; 10 (MW&#8805;136), which calls for development of a model that works for smaller SCN groups. I developed a simple 3rd degree polynomial that fits SG<sub>liq</sub> of lower SCN groups well for MW<136 in <a href="#eq-3" class="internal-link">Eq-3</a>. Accuracy of the model performance can be visually evaluated in <a href="#fig-13" class="internal-link">Figure 13</a>.

<div id="eq-3" style="font-size: 1rem;">
$$ SG_{liq} = (3.018 \times 10^{-7})MW^3 - (1.214 \times 10^{-4})MW^2 + 0.01719MW - 0.06947  \space  \space  \space  \space  \space  \text{for MW<136} \tag{3}$$
</div>

Combining <a href="#eq-2" class="internal-link">Eq-2</a> and <a href="#eq-3" class="internal-link">Eq-3</a> gives a piece-wise function shown in <a href="#eq-4" class="internal-link">Eq-4</a>, which has a working range between 6&lt;SCN&lt;50 (82&lt;MW&lt;698). Check <a href="#fig-14" class="internal-link">Figure 14</a> for visual inspection of the final model.

<div id="eq-4" style="font-size: 1rem;">
$$ SG_{liq}=   \left\{
    \begin{array}{ll}
          (3.018 \times 10^{-7})MW^3 - (1.214 \times 10^{-4})MW^2 + 0.01719MW - 0.06947 & \text{for } MW < 136,  \\
           1.07 - \text{exp}(3.56073 - 2.93886MW^{0.1}) &  \text{for } MW \geq 136  \\
    \end{array} 
    \right.  
    \tag{4}$$
</div>

<a href="#table-2" class="internal-link">Table 2</a> provides a tabulated MW-SG<sub>liq</sub> data generated with <a href="#eq-4" class="internal-link">Eq-4</a> for quick lookup of gas samples between 80&#8804;MW&#8804;136. Since the max plus fraction reported for gas samples is Decanes+, SCN=10 with MW=136 should cover all observable ranges of gas plus fractions. 

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-12">
<div class="col"><img src="jupyter_images/liquid_sg_scn_model_riazi_original2.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 12:</strong> Comparison of SCN models and normal paraffins. Real-life plus fractions contain naphthenic and aromationc fractions, which are known to have higher SG<sub>liq</sub> than paraffins at a given MW, as shown in <a href="#fig-2" class="internal-link">Figure 2</a> above. The values of normal paraffins are obtained with <a href="#code_snippet_1" class="internal-link">Code Snippet #1</a>.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (12)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('scn_table.csv')

MW_1 = df['MW [K&F]']
SG_1 = df['SG [K&F]']
Tb_1 = df['Tb [R] [K&F]']

MW_2 = df['MW [R&A]']
SG_2 = df['SG [R&A]']
Tb_2 = df['Tb [R] [R&A]']

MW_3 = df['MW [Normal Paraffins]']
SG_3 = df['SG [Normal Paraffins]']
Tb_3 = df['Tb [R] [Normal Paraffins]']

def calc_sg_liq_riazi(mw):
    return 1.07 - np.exp(3.56073 - 2.93886 * mw**0.1)

x = np.arange(80, 500, 2)
y_riazi = np.array([calc_sg_liq_riazi(mw) for mw in x])

fig, ax = plt.subplots(figsize=(8, 4.5))

line1, = ax.plot(x, y_riazi, label='Riazi & Al-Sahhaf (1996): $SG_{liq}=a_1-exp(a_2 - a_3 \cdot MW^{a_4})$')

scatter1 = ax.scatter(MW_1, SG_1, label='Katz & Firoozabadi (1978)', marker='d', c='red')
scatter2 = ax.scatter(MW_2, SG_2, label='Riazi & Al-Sahhaf (1996)', marker='+', c='navy', s=100)
scatter3 = ax.scatter(MW_3, SG_3, label='Normal paraffins', marker='v', c='k')

legend1 = ax.legend(handles=[scatter1, scatter2, scatter3], fontsize=10, ncol=3, loc='upper left')
ax.add_artist(legend1)
ax.legend(handles=[line1], loc='lower right', fontsize=10)


ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Molecular Weight', fontsize=13)
ax.set_ylabel('Liquid Specific Gravity', fontsize=13)
ax.text(0.02, 0.05, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

ax.text(0.7, 0.23, '$a_1$=1.07', fontsize=10, ha='left', va='center', transform=ax.transAxes)
ax.text(0.7, 0.17, '$a_2$=3.56073', fontsize=10, ha='left', va='center', transform=ax.transAxes)
ax.text(0.85, 0.23, '$a_3$=2.93886', fontsize=10, ha='left', va='center', transform=ax.transAxes)
ax.text(0.85, 0.17, '$a_4$=0.1', fontsize=10, ha='left', va='center', transform=ax.transAxes)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('MW vs. SG_{liq} Correlation by Riazi')
plain_txt = r', for SCN groups'
fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0, horizontalalignment='left', fontsize=13, y=0.96)
yloc = 0.88
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.02, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

ax.set_xlim(50, 450)
ax.set_ylim(0.6, .9999)

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-13">
<div class="col"><img src="jupyter_images/liquid_sg_scn_model_new_fit.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 13:</strong> 3rd degree polynomial fit with <a href="#eq-3" class="internal-link">Eq-3</a> for MW &lt; 136. This new model fits lower MW data points better than Riazi's original model (<a href="#eq-2" class="internal-link">Eq-2</a>).</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (13)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('scn_table.csv')

MW_1 = df['MW [K&F]']
SG_1 = df['SG [K&F]']
Tb_1 = df['Tb [R] [K&F]']

MW_2 = df['MW [R&A]']
SG_2 = df['SG [R&A]']
Tb_2 = df['Tb [R] [R&A]']

MW_3 = df['MW [Normal Paraffins]']
SG_3 = df['SG [Normal Paraffins]']
Tb_3 = df['Tb [R] [Normal Paraffins]']

def calc_sg_liq_upto_136(mw):
    return 3.0184e-7 * mw**3 - 1.2139e-4 * mw**2 + 1.7187e-2 * mw - 6.9473e-2

x = np.arange(80, 138, 2)
y_136 = np.array([calc_sg_liq_upto_136(mw) for mw in x])

fig, ax = plt.subplots(figsize=(8, 4.5))

line1, = ax.plot(x, y_136, label='New Fit for MW < 136: Eq-3', c='#ff7f0e')

scatter1 = ax.scatter(MW_1, SG_1, label='Katz & Firoozabadi (1978)', marker='d', c='red')
scatter2 = ax.scatter(MW_2, SG_2, label='Riazi & Al-Sahhaf (1996)', marker='+', c='navy', s=100)
scatter3 = ax.scatter(MW_3, SG_3, label='Normal paraffins', marker='v', c='k')

legend1 = ax.legend(handles=[scatter1, scatter2, scatter3], fontsize=10, ncol=3, loc='upper left')
ax.add_artist(legend1)
ax.legend(handles=[line1], loc='lower right', fontsize=10)


ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Molecular Weight', fontsize=13)
ax.set_ylabel('Liquid Specific Gravity', fontsize=13)
ax.text(0.02, 0.05, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)



def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('Modified MW vs. SG_{liq} Correlation')
plain_txt = r', for MW < 136'
fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0, horizontalalignment='left', fontsize=13, y=0.96)
yloc = 0.88
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.02, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

ax.set_xlim(79.5, 140)
ax.set_ylim(0.68001, .7999)

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-14">
<div class="col"><img src="jupyter_images/liquid_sg_scn_piecewise_model.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 14:</strong> Piecewise function from <a href="#eq-4" class="internal-link">Eq-4</a>. Note that while the function can be extended infinitely, the model has been tested only for 82 &lt; MW &lt; 698. Extrapolation beyond this tested range should be performed with caution.</p></div>

<div class="solution_panel closed" style="margin-top: -10px">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (14)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('scn_table.csv')

MW_1 = df['MW [K&F]']
SG_1 = df['SG [K&F]']
Tb_1 = df['Tb [R] [K&F]']

MW_2 = df['MW [R&A]']
SG_2 = df['SG [R&A]']
Tb_2 = df['Tb [R] [R&A]']

MW_3 = df['MW [Normal Paraffins]']
SG_3 = df['SG [Normal Paraffins]']
Tb_3 = df['Tb [R] [Normal Paraffins]']

def calc_sg_liq_riazi(mw):
    return 1.07 - np.exp(3.56073 - 2.93886 * mw**0.1)

def calc_sg_liq_modified(mw):
    return 3.0184e-7 * mw**3 - 1.2139e-4 * mw**2 + 1.7187e-2 * mw - 6.9473e-2

def calc_sg_liq_piecewise(mw):
    if mw < 136:
        return calc_sg_liq_modified(mw)
    else:
        return calc_sg_liq_riazi(mw)

x = np.arange(60, 500, 2)
y = np.array([calc_sg_liq_piecewise(mw) for mw in x])

fig, ax = plt.subplots(figsize=(8, 4.5))

line1, = ax.plot(x, y, label='Piecewise model: Eq-4', c='#2ca02c')

scatter1 = ax.scatter(MW_1, SG_1, label='Katz & Firoozabadi (1978)', marker='d', c='red')
scatter2 = ax.scatter(MW_2, SG_2, label='Riazi & Al-Sahhaf (1996)', marker='+', c='navy', s=100)
scatter3 = ax.scatter(MW_3, SG_3, label='Normal paraffins', marker='v', c='k')

legend1 = ax.legend(handles=[scatter1, scatter2, scatter3], fontsize=10, ncol=3, loc='upper left')
ax.add_artist(legend1)
ax.legend(handles=[line1], loc='lower right', fontsize=10)


ax.minorticks_on()
ax.grid(axis='y', which='major', linestyle='--', color='grey', alpha=0.5)
ax.grid(axis='x', which='major', color='grey', linestyle='--', alpha=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Molecular Weight', fontsize=13)
ax.set_ylabel('Liquid Specific Gravity', fontsize=13)
ax.text(0.02, 0.05, 'aegis4048.github.io', fontsize=12, ha='left', va='center',
    transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt):
    return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])

bold_txt = setbold('Piecewise Modified MW vs. SG_{liq} Correlation')
plain_txt = r', for SCN groups'
fig.suptitle(bold_txt + plain_txt, verticalalignment='top', x=0, horizontalalignment='left', fontsize=13, y=0.96)
yloc = 0.88
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction', xytext=(1.02, yloc + 0.01),
            arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

ax.set_xlim(50, 450)
ax.set_ylim(0.6, .9999)

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="row full_screen_margin_75 mobile_responsive_plot_full_width" id="table-1">
<div class="col"><img src="jupyter_images/liquid_sg_scn_tables2.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Table 1:</strong> SCN property tables featuring Katz &amp; Firoozabadi (1978)<sup><a class="internal-link" href="#id4" id="id42">[4]</a></sup>, Riazi &amp; Al-Sahhaf (1996)<sup><a class="internal-link" href="#id5" id="id52">[5]</a></sup>, and normal paraffins. The properites of normal paraffins are extracted with <a href="#code_snippet_1" class="internal-link">Code Snippet #1</a>. Note that the liquid specific gravity for normal paraffins is omitted beyond C16 because the python library used to pull the data reports densites at 60F, which suffers inconsistencies due to paraffins beyond C16 existing as solids at 60F.</p></div>

<div><hr></div>

<strong id="code_snippet_1">Code Snippet #1</strong>

In [6]:
from thermo import ChemicalConstantsPackage  # pip install thermo
import numpy as np
import pint                                  # pip install pint


ureg = pint.UnitRegistry()

density_water = 999.0170125317171, # (kg/m^3) @60F-1atm according to IAPWS-95 standard, chemicals.iapws95_rho(288.706, 101325)

n_paraffins = ['n-C6', 'n-C7', 'n-C8', 'n-C9', 'n-C10',
 'n-C11', 'n-C12', 'n-C13', 'n-C14', 'n-C15', 'n-C16', 'n-C17', 'n-C18', 'n-C19', 'n-C20',
 'n-C21', 'n-C22', 'n-C23', 'n-C24', 'n-C25', 'n-C26', 'n-C27', 'n-C28', 'n-C29', 'n-C30',
 'n-C31', 'n-C32', 'n-C33', 'n-C34', 'n-C35', 'n-C36', 'n-C37', 'n-C38', 'n-C39', 'n-C40',
 'n-C41', 'n-C42', 'n-C43', 'n-C44', 'n-C45', 'n-C46', 'n-C47', 'n-C48', 'n-C49', 'n-C50']

constants = ChemicalConstantsPackage.constants_from_IDs(n_paraffins)
MWs = np.array(constants.MWs)
sgs = np.array(constants.rhol_60Fs_mass) / density_water  # values are invalid after C16 because they are maybe solids at 60F
Tbs = np.array([ureg('%.15f kelvin' % Tb).to('rankine')._magnitude for Tb in np.array(constants.Tbs)])

<div class="row full_screen_margin_90 mobile_responsive_plot_full_width" id="table-2">
<div class="col"><img src="jupyter_images/liquid_sg_scn_tabulated_scn.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Table 2:</strong> Tablulated MW-SG<sub>liq</sub> correlation values for quick lookup using <a href="#eq-4" class="internal-link">Eq-4</a>. The liquid specific gravity values are generated with <a href="#code_snippet_2" class="internal-link">Code Snippet #2</a>. This table should work for all practical modeling of gas samples, since the heaviest plus fraction you will see in gas samples is Decanes+ (SCN=10, MW=136).</p></div>

<div><hr></div>

<strong id="code_snippet_2">Code Snippet #2</strong>

In [8]:
import numpy as np
import pint

ureg = pint.UnitRegistry()

def calc_sg_liq_riazi(mw):
    return 1.07 - np.exp(3.56073 - 2.93886 * mw**0.1)

def calc_sg_liq_modified(mw):
    return 3.0184e-7 * mw**3 - 1.2139e-4 * mw**2 + 1.7187e-2 * mw - 6.9473e-2

def calc_sg_liq_piecewise(mw):
    if mw < 136:
        return calc_sg_liq_modified(mw)
    else:
        return calc_sg_liq_riazi(mw)
    
def calc_Tb(mw):
    Tb = 1080 - np.exp(6.97996 - 0.01964 * mw**(2/3))
    return ureg('%.15f kelvin' % Tb).to('rankine')._magnitude

MWs = np.arange(80, 137, 1)
sgs = np.array([calc_sg_liq_piecewise(mw) for mw in MWs])
Tbs = np.array([calc_Tb(mw) for mw in MWs])

## 5. Excercises

The purpose of the following exercises are to approximate liquid specific gravities (SG<sub>liq</sub>) of gas sample analyses, when then lab reports are missing values of SG<sub>liq</sub>, or making certain assumptions about the composition of the plus fractions (such as 6:3:1 ratio of C6, C7, and C8) instead of performing physical measurements.

### 5.1. Example 1

The sample analysis is given in <a href="#fig-15" class="internal-link">Figure 15</a>. The lab report is a 2-pager extended analysis, with the 2nd page reporting 18 compounds between n-hexane and n-decane. We ignore the "Undecane (11)" for now. 

Realistically the <i>Heptanes Plus</i> fraction can be replaced with simple 6:3:1 ternary mixture of n-hexane, n-heptane, and n-octane since it makes up only 0.344 mol% of the whole sample. The error from the ternary mixture assumption will be negligible. But for the sake of learning, let's assume that we actually care about this. 

At a first glance, it seems that we can simply enter all compounds into a simulation software instead of characterizing a plus fraction, since the extended analysis reports all measured compound mol %. However, taking a second look at it, you can notice some ambiguous compounds such as [Other C7's, Other C8's, Other C8's, Other C9's, Other C10's]. Unfortunately the lab report didn't provide us with detailed definition of the "Other C<sub>n</sub>'s", forcing us to actually characterize the Heptanes Plus fraction.

The lab probably has their own definition of the "Other C<sub>n</sub>'s", and reported MW=95.59 for the plus fraction based on their own definition. Given MW=95.59, SG<sub>liq</sub> can be calculated from <a href="#eq-3" class="internal-link">Eq-3</a>, or can be looked up from <a href="#table-2" class="internal-link">Table 2</a>, giving SG<sub>liq</sub>=0.7279. The sample now has both values of MW and SG<sub>liq</sub> and can be modeled in a simulation software, as in <a href="#fig-16" class="internal-link">Figure 16</a>

<div class="row full_screen_margin_80 mobile_responsive_plot_full_width" id="fig-15">
<div class="col"><img src="jupyter_images/liquid_sg_excercise_1.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 15:</strong> Example gas analysis 1.</p></div>

<div class="row full_screen_margin_70 mobile_responsive_plot_full_width" id="fig-16">
<div class="col"><img src="jupyter_images/liquid_sg_excercise_1_promax.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 16:</strong> Promax simulation result for the example gas analysis 1.</p></div>

### 5.2. Example 2

The sample analysis is given in <a href="#fig-17" class="internal-link">Figure 17</a>. This particular sample is a natural gas liquid (NGL) taken from a pressurized stream. Operators sometimes choose to sell the rich vapors in a liquid phase from the gas outlet of low pressure vessels such as heater treater (HT), vapor recovery tower (VRT) or atmospheric tanks because NGLs are sold for a higher price than the equivalent MMBtu gas. Usually the NGLs are sold for half the price of crude oils. The rich vapors can be knocked out as NGLs with compression, and stored in pressurized containers. 

This sample analysis is provided from the same laboratory as in <a href="#fig-10" class="internal-link">Figure 10</a> above. The difference is that the reported Hexanes Plus properties of this sample are not measured, but assumed. There are two red flags that should raise your eyebrows as you are evaluating this analysis report.

First, the average MW=93.189 can be obtained if you assume 60% n-hexane (MW=86.17), 30% n-heptane (MW=100.21), 10% n-octane (MW=114.23). Their weighted average is 93.188, which matches exactly with the provided mixture MW. This is too convenient to be true if they actually made physical measurements.

Second, notice the small font "Page 1 of 1" at the bottom right corner. If this was an extended analysis, this should be at least a 2-page report in which the 2nd page has detailed measured mole fractions of heavier compounds. If this is not an extended analysis, it means that the Hexanes Plus fraction is not measured.

As shown in <a href="#fig-2" class="internal-link">Figure 2</a> and <a href="#fig-12" class="internal-link">Figure 12</a>, assuming 100% paraffinic composition results in underestimated SG<sub>liq</sub> for the plus fraction due to contributions from naphthenic and aromatic fractions. It also overestimates MW, as shown in <a href="#table-2" class="internal-link">Table 2</a>. C6 from the SCN model has MW=82, but the equivalent paraffin (n-hexane, C6H14) has MW=86.18. <strong><u>This is a significant source of error because the Hexanes Plus fraction make up 60% of the whole sample. The engineer who asked for this sample analysis really should've asked for extended analysis.</u></strong>

But this is how life works, and we gotta deal with what we got. Unfortunately for this kind of analysis we gotta make assumptions based on our previous experience. The following is the brainstorming process:


<div class="ordered-list">
    <h2>Brainstorming process</h2>
    <ol>
        <li>This sample came from a pressurized NGL line.</li>
        <li>Usually the NGL's are made with rich vapors from HT, VRT, or atmospheric tanks.</li>
        <li>Compression is required, which is usually done with VRUs (screw or recip compressors).</li>
        <li>The gas sample analysis I've seen from VRU inlets or outlets usually have MW range between 90 ~ 100.</li>
        <li>Potential MW range of 90~100 is the best I've got, I will pick the middle point of MW=95.</li>
        <li>Quick lookup on <a href="#table-2" class="internal-link">Table 2</a> indicates that a fraction with MW=95 should have SG<sub>liq</sub>=0.727.</li>
    </ol>
</div>

Note that the above brainstorming procedure is based on my personal experience and therefore completely subjective. However, * believe it's the best possible approximation with the quality of data we have. The Hexanes Plus fraction now has both values of MW=95 and SG<sub>liq</sub>=0.727 and can be modeled in a simulation software, as in <a href="#fig-18" class="internal-link">Figure 18</a>.

<div class="row full_screen_margin_90 mobile_responsive_plot_full_width" id="fig-17">
<div class="col"><img src="jupyter_images/liquid_sg_excercise_2.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 17:</strong> Example gas analysis 2.</p></div>

<div class="row full_screen_margin_90 mobile_responsive_plot_full_width" id="fig-18">
<div class="col"><img src="jupyter_images/liquid_sg_excercise_2_promax.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 18:</strong> Promax simulation result for the example gas analysis 2.</p></div>

### 5.3. Example 3

The sample analysis is given in <a href="#fig-19" class="internal-link">Figure 19</a>.

<div class="row full_screen_margin_90 mobile_responsive_plot_full_width" id="fig-19">
<div class="col"><img src="jupyter_images/liquid_sg_excercise_3.png" style="margin-bottom: 10px;"></div></div>    
<div class="col-12 fig-title"><p class="image-description"><strong>Figure 19:</strong> Example gas analysis 3.</p></div>

## 6. References

<span id="id1" ><strong>[1](<a class="internal-link" href="#id11" style="content: none;">1</a>)</strong></span> Riazi, M. R.: "Characterization and Properties of Petroleum Fractions," (2005), West Conshohocken, Pennsylvania: ASTM International

<span id="id2"><strong>[2](<a class="internal-link" href="#id21" style="content: none;">1</a>, <a class="internal-link" href="#id22" style="content: none;">2</a>, <a class="internal-link" href="#id23" style="content: none;">3</a>, <a class="internal-link" href="#id24" style="content: none;">4</a>)</strong></span> GPA Mistream Association: "GPA Midstream Standard 2145, Table of Physical Properties for Hydrocarbons and Other Compounds of Interest to the Natural Gas and Natural Gas Liquids Industries" (2016)

<span id="id3"><strong>[3](<a class="internal-link" href="#id31" style="content: none;">1</a>)</strong></span> GPA Mistream Association: "GPA Midstream Standard 2286, Tentative Method of Extended Analysis for Natural Gas and Similar Gaseous Mixtures by Temperature Programmed Gas Chromatography" (1995)

<span id="id4"><strong>[4](<a class="internal-link" href="#id41" style="content: none;">1</a>, <a class="internal-link" href="#id42" style="content: none;">2</a>)</strong></span> Katz, D.L., Firoozabadi, A.: "Predicting Phase Behavior of Condensate/Crude-Oil Systems Using Methane Interaction Coefficients" (1978)

<span id="id5"><strong>[5](<a class="internal-link" href="#id51" style="content: none;">1</a>, <a class="internal-link" href="#id52" style="content: none;">2</a>)</strong></span> Riazi, M.R., and Al-Sahhaf, T.A.: "Physical Properties of Heavy Petroleum Fractions and Crude Oils" (1996)


<span id="id3"><strong>[3](<a class="internal-link" href="#id31" style="content: none;">1</a>)</strong></span> Riazi, M.R., and Al-Sahhaf, T.A.: "Physical Properties of Heavy Petroleum Fractions and Crude Oils" (1996), Fluid Phase Equilibria 117.

<span id="id4"><strong>[4](<a class="internal-link" href="#id41" style="content: none;">1</a>)</strong></span> Speight, J. G.: "The Chemistry and Technology of Petroleum," 3rd ed., Marcel Dekker, New York, 1999
